In [3]:
from dagster_pipeline.dagster_acled.resources.resources import ResourceConfig
from dagster_pipeline.dagster_acled.secrets_config import SecretManager

import os

sm = SecretManager(region_name=os.environ['REGION_NAME'])
resources = ResourceConfig.from_secrets(sm = sm, s3_secret_name="acled_bucket", pg_secret_name="acled_postgres")

In [5]:
conn = resources['postgres'].get_connection()

In [7]:
import pandas as pd

query = "SELECT * FROM acled_events_no_delete"

quality_df = pd.read_sql_query(query, conn)

/var/folders/rs/lp_7j4vj5_s59_pr61yffxl80000gn/T/ipykernel_51987/259727756.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quality_df = pd.read_sql_query(query, conn)


In [12]:
df = quality_df[quality_df['fatalities'] != 0]

In [20]:
pd.set_option("display.max_columns", None)

In [22]:
df = df[['disorder_type', 'event_type', 'sub_event_type', 'actor1', 'inter1', 'actor2', 'inter2', 'interaction', 'admin1', 'admin2', 'admin3', 'latitude', 'longitude', 'fatalities']]
df.head()

,disorder_type,event_type,sub_event_type,actor1,inter1,actor2,inter2,interaction,admin1,admin2,admin3,latitude,longitude,fatalities
11,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,External/Other forces,Military Forces of Ukraine (2019-),State forces,State forces-External/Other forces,Zaporizhia,Polohivskyi,Huliaipilska,47.6639,36.2563,2
16,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,External/Other forces,Military Forces of Ukraine (2019-),State forces,State forces-External/Other forces,Donetsk,Kramatorskyi,Kostiantynivska,48.5333,37.7166,1
24,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,External/Other forces,Military Forces of Ukraine (2019-),State forces,State forces-External/Other forces,Donetsk,Pokrovskyi,Hrodivska,48.2986,37.3804,2
34,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,External/Other forces,Military Forces of Ukraine (2019-),State forces,State forces-External/Other forces,Zaporizhia,Vasylivskyi,Stepnohirska,47.5079,35.4622,1
36,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,External/Other forces,Military Forces of Ukraine (2019-),State forces,State forces-External/Other forces,Donetsk,Pokrovskyi,Dobropilska,48.3823,37.1096,2


In [25]:
for col in df.columns: 
    print(f'{col}:')
    print(df[col].value_counts())
    print("-" * 80)

disorder_type:
disorder_type
Political violence        9606
Strategic developments       2
Name: count, dtype: int64
--------------------------------------------------------------------------------
event_type:
event_type
Battles                       5187
Explosions/Remote violence    4407
Violence against civilians      12
Strategic developments           2
Name: count, dtype: int64
--------------------------------------------------------------------------------
sub_event_type:
sub_event_type
Armed clash                            5150
Shelling/artillery/missile attack      2979
Air/drone strike                       1379
Remote explosive/landmine/IED            49
Non-state actor overtakes territory      29
Attack                                   12
Government regains territory              8
Disrupted weapons use                     1
Other                                     1
Name: count, dtype: int64
-------------------------------------------------------------------------------

In [26]:
df['actor2'] = df['actor2'].replace('', pd.NA)
df['inter2'] = df['inter2'].replace('', pd.NA)

In [28]:
import pandas as pd
import numpy as np

def inspect_missing_values(df):
    """
    Comprehensive inspection of missing values and empty strings in DataFrame
    """
    print("="*80)
    print("MISSING VALUES & EMPTY STRING INSPECTION")
    print("="*80)
    
    # Get basic info about the dataset
    print(f"Dataset shape: {df.shape}")
    print(f"Total cells: {df.shape[0] * df.shape[1]:,}")
    print()
    
    # Initialize results dictionary
    results = {}
    
    print("COLUMN-BY-COLUMN ANALYSIS:")
    print("-" * 50)
    
    for col in df.columns:
        print(f"\n📊 Column: {col}")
        print(f"   Data type: {df[col].dtype}")
        
        # Count different types of missing/empty values
        na_count = df[col].isna().sum()
        null_count = df[col].isnull().sum()
        
        # Check for empty strings (only for object/string columns)
        if df[col].dtype == 'object':
            empty_str_count = (df[col] == '').sum()
            whitespace_count = df[col].str.strip().eq('').sum() - empty_str_count
            none_str_count = (df[col] == 'None').sum()
        else:
            empty_str_count = 0
            whitespace_count = 0
            none_str_count = 0
        
        # Total missing
        total_missing = na_count + empty_str_count + whitespace_count
        missing_pct = (total_missing / len(df)) * 100
        
        # Store results
        results[col] = {
            'dtype': str(df[col].dtype),
            'na_count': na_count,
            'empty_strings': empty_str_count,
            'whitespace_only': whitespace_count,
            'none_strings': none_str_count,
            'total_missing': total_missing,
            'missing_percentage': missing_pct,
            'unique_values': df[col].nunique(),
            'total_values': len(df)
        }
        
        # Print summary for this column
        print(f"   Total values: {len(df):,}")
        print(f"   Unique values: {df[col].nunique():,}")
        print(f"   NA/NaN values: {na_count:,}")
        
        if df[col].dtype == 'object':
            print(f"   Empty strings (''): {empty_str_count:,}")
            if whitespace_count > 0:
                print(f"   Whitespace only: {whitespace_count:,}")
            if none_str_count > 0:
                print(f"   'None' strings: {none_str_count:,}")
        
        print(f"   Total missing: {total_missing:,} ({missing_pct:.2f}%)")
        
        # Show examples of missing values if any exist
        if total_missing > 0:
            print(f"   ⚠️  HAS MISSING VALUES!")
            
            # Show sample of actual values to understand the pattern
            if df[col].dtype == 'object':
                value_counts = df[col].value_counts(dropna=False, normalize=True).head(5)
                print(f"   Top values:")
                for val, pct in value_counts.items():
                    display_val = repr(val) if pd.isna(val) or val == '' else str(val)
                    print(f"     {display_val}: {pct:.1%}")
        else:
            print(f"   ✅ No missing values")
    
    # Summary table
    print("\n" + "="*80)
    print("SUMMARY TABLE")
    print("="*80)
    
    summary_df = pd.DataFrame(results).T
    summary_df = summary_df.sort_values('missing_percentage', ascending=False)
    
    print(summary_df[['dtype', 'total_missing', 'missing_percentage', 'empty_strings', 'na_count']].round(2))
    
    # Overall statistics
    print(f"\n📈 OVERALL STATISTICS:")
    print("-" * 30)
    total_cells = df.shape[0] * df.shape[1]
    total_missing_cells = sum(results[col]['total_missing'] for col in results)
    overall_missing_pct = (total_missing_cells / total_cells) * 100
    
    print(f"Total cells in dataset: {total_cells:,}")
    print(f"Total missing cells: {total_missing_cells:,}")
    print(f"Overall missing percentage: {overall_missing_pct:.2f}%")
    
    # Columns with issues
    problematic_cols = [col for col in results if results[col]['total_missing'] > 0]
    print(f"Columns with missing values: {len(problematic_cols)} out of {len(df.columns)}")
    
    if problematic_cols:
        print(f"Problematic columns: {', '.join(problematic_cols)}")
    
    # Special attention to empty strings
    empty_string_cols = [col for col in results if results[col]['empty_strings'] > 0]
    if empty_string_cols:
        print(f"\n⚠️  Columns with empty strings: {', '.join(empty_string_cols)}")
    
    return results

def detailed_empty_string_analysis(df, columns_to_check=None):
    """
    Deep dive into empty string patterns
    """
    if columns_to_check is None:
        columns_to_check = df.select_dtypes(include=['object']).columns
    
    print("\n" + "="*80)
    print("DETAILED EMPTY STRING ANALYSIS")
    print("="*80)
    
    for col in columns_to_check:
        if (df[col] == '').any():
            print(f"\n🔍 Column: {col}")
            
            # Show context around empty strings
            empty_indices = df[df[col] == ''].index[:5]  # First 5 empty string rows
            
            print(f"   Rows with empty strings (showing first 5):")
            for idx in empty_indices:
                row_data = df.loc[idx, [col] + [c for c in df.columns if c != col][:3]]
                print(f"   Row {idx}: {dict(row_data)}")

# Usage example:
# results = inspect_missing_values(df)
# detailed_empty_string_analysis(df)

# Quick check function for immediate use
def quick_missing_check(df):
    """Quick overview of missing values"""
    print("QUICK MISSING VALUES CHECK:")
    print("-" * 40)
    
    for col in df.columns:
        na_count = df[col].isna().sum()
        empty_count = (df[col] == '').sum() if df[col].dtype == 'object' else 0
        total_missing = na_count + empty_count
        
        if total_missing > 0:
            pct = (total_missing / len(df)) * 100
            print(f"{col}: {total_missing:,} missing ({pct:.1f}%) - NA: {na_count}, Empty: {empty_count}")
    
    print(f"\nTotal rows: {len(df):,}")

#Run this on your dataframe:
quick_missing_check(df)
results = inspect_missing_values(df)

QUICK MISSING VALUES CHECK:
----------------------------------------
actor2: 4 missing (0.0%) - NA: 4, Empty: 0
inter2: 4 missing (0.0%) - NA: 4, Empty: 0
admin3: 24 missing (0.2%) - NA: 0, Empty: 24

Total rows: 9,608
MISSING VALUES & EMPTY STRING INSPECTION
Dataset shape: (9608, 14)
Total cells: 134,512

COLUMN-BY-COLUMN ANALYSIS:
--------------------------------------------------

📊 Column: disorder_type
   Data type: object
   Total values: 9,608
   Unique values: 2
   NA/NaN values: 0
   Empty strings (''): 0
   Total missing: 0 (0.00%)
   ✅ No missing values

📊 Column: event_type
   Data type: object
   Total values: 9,608
   Unique values: 4
   NA/NaN values: 0
   Empty strings (''): 0
   Total missing: 0 (0.00%)
   ✅ No missing values

📊 Column: sub_event_type
   Data type: object
   Total values: 9,608
   Unique values: 9
   NA/NaN values: 0
   Empty strings (''): 0
   Total missing: 0 (0.00%)
   ✅ No missing values

📊 Column: actor1
   Data type: object
   Total values: 9,608

In [29]:
# Analyze cardinality to choose encoding methods
def analyze_categorical_features(df, target_col='fatalities'):
    """Analyze categorical features to determine best encoding approach"""
    
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    print("CATEGORICAL FEATURE ANALYSIS:")
    print("-" * 50)
    
    encoding_strategy = {}
    
    for col in categorical_cols:
        unique_count = df[col].nunique()
        total_rows = len(df)
        cardinality_ratio = unique_count / total_rows
        
        # Calculate target correlation (for target encoding decision)
        grouped = df.groupby(col)[target_col].agg(['mean', 'count'])
        target_variance = grouped['mean'].var()
        
        print(f"\n{col}:")
        print(f"  Unique values: {unique_count}")
        print(f"  Cardinality ratio: {cardinality_ratio:.3f}")
        print(f"  Target variance: {target_variance:.3f}")
        
        # Determine encoding strategy
        if unique_count <= 10:
            strategy = "One-Hot Encoding"
        elif unique_count <= 50 and cardinality_ratio < 0.5:
            strategy = "Label Encoding or Target Encoding"
        elif target_variance > 1.0:  # High target correlation
            strategy = "Target Encoding (Recommended)"
        else:
            strategy = "Frequency/Count Encoding"
            
        encoding_strategy[col] = strategy
        print(f"  Recommended: {strategy}")
    
    return encoding_strategy

# Run this analysis
encoding_plan = analyze_categorical_features(df)

CATEGORICAL FEATURE ANALYSIS:
--------------------------------------------------

disorder_type:
  Unique values: 2
  Cardinality ratio: 0.000
  Target variance: 6.636
  Recommended: One-Hot Encoding

event_type:
  Unique values: 4
  Cardinality ratio: 0.000
  Target variance: 9.297
  Recommended: One-Hot Encoding

sub_event_type:
  Unique values: 9
  Cardinality ratio: 0.001
  Target variance: 34.623
  Recommended: One-Hot Encoding

actor1:
  Unique values: 20
  Cardinality ratio: 0.002
  Target variance: 146.612
  Recommended: Label Encoding or Target Encoding

inter1:
  Unique values: 4
  Cardinality ratio: 0.000
  Target variance: 160.570
  Recommended: One-Hot Encoding

actor2:
  Unique values: 24
  Cardinality ratio: 0.002
  Target variance: 133.008
  Recommended: Label Encoding or Target Encoding

inter2:
  Unique values: 4
  Cardinality ratio: 0.000
  Target variance: 313.236
  Recommended: One-Hot Encoding

interaction:
  Unique values: 14
  Cardinality ratio: 0.001
  Target v

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

def target_encode_cv(df, categorical_col, target_col, cv_folds=5, smoothing=10):
    """
    Target encoding with cross-validation and smoothing to prevent overfitting
    """
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    target_encoded = np.zeros(len(df))
    
    # Global mean for smoothing and fallback
    global_mean = df[target_col].mean()
    
    for train_idx, val_idx in kf.split(df):
        # Calculate target statistics on training fold
        train_df = df.iloc[train_idx]
        target_stats = train_df.groupby(categorical_col)[target_col].agg(['mean', 'count'])
        
        # Apply smoothing: (count * category_mean + smoothing * global_mean) / (count + smoothing)
        target_stats['smoothed_mean'] = (
            (target_stats['count'] * target_stats['mean'] + smoothing * global_mean) /
            (target_stats['count'] + smoothing)
        )
        
        # Apply to validation fold
        target_encoded[val_idx] = df.iloc[val_idx][categorical_col].map(target_stats['smoothed_mean'])
        
        # Handle unseen categories with global mean
        target_encoded[val_idx] = np.where(
            pd.isna(target_encoded[val_idx]), 
            global_mean, 
            target_encoded[val_idx]
        )
    
    return target_encoded

def prepare_xgboost_features(df, target_col='fatalities'):
    """
    Complete preprocessing pipeline for XGBoost based on your analysis
    """
    print("🚀 Starting XGBoost preprocessing pipeline...")
    df_processed = df.copy()
    
    # =============================================================================
    # 1. ONE-HOT ENCODING (Low cardinality, high target variance)
    # =============================================================================
    onehot_cols = [
        'disorder_type',    # 2 unique, variance: 6.636
        'event_type',       # 4 unique, variance: 9.297  
        'sub_event_type',   # 9 unique, variance: 34.623
        'inter1',           # 4 unique, variance: 160.570
        'inter2'            # 4 unique, variance: 313.236
    ]
    
    print(f"📊 Applying One-Hot Encoding to: {onehot_cols}")
    df_encoded = pd.get_dummies(df_processed, columns=onehot_cols, prefix=onehot_cols)
    print(f"   Created {len([col for col in df_encoded.columns if any(prefix in col for prefix in onehot_cols)])} binary features")
    
    # =============================================================================
    # 2. TARGET ENCODING (High cardinality)
    # =============================================================================
    target_encode_cols = [
        'admin2',  # 71 unique, variance: 8.676
        'admin3'   # 236 unique, variance: 12.173
    ]
    
    print(f"🎯 Applying Target Encoding to: {target_encode_cols}")
    for col in target_encode_cols:
        encoded_col_name = f'{col}_target_encoded'
        df_encoded[encoded_col_name] = target_encode_cv(df_encoded, col, target_col)
        print(f"   ✓ {col} → {encoded_col_name}")
    
    # =============================================================================
    # 3. DECISION FOR MEDIUM CARDINALITY (Manual choice needed)
    # =============================================================================
    medium_cardinality_cols = {
        'actor1': {'unique': 20, 'variance': 146.612},
        'actor2': {'unique': 24, 'variance': 133.008}, 
        'interaction': {'unique': 14, 'variance': 1.608},
        'admin1': {'unique': 20, 'variance': 14.802}
    }
    
    print(f"⚖️  Processing Medium Cardinality columns:")
    
    # HIGH variance → Target Encoding
    high_variance_cols = ['actor1', 'actor2']  # variance > 100
    for col in high_variance_cols:
        encoded_col_name = f'{col}_target_encoded'
        df_encoded[encoded_col_name] = target_encode_cv(df_encoded, col, target_col)
        print(f"   🎯 {col} → Target Encoded (high variance: {medium_cardinality_cols[col]['variance']:.1f})")
    
    # LOWER variance → Label Encoding (ordinal, preserves some relationship)
    lower_variance_cols = ['interaction', 'admin1']  # variance < 20
    label_encoders = {}
    for col in lower_variance_cols:
        le = LabelEncoder()
        encoded_col_name = f'{col}_label_encoded'
        df_encoded[encoded_col_name] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le
        print(f"   🏷️  {col} → Label Encoded (lower variance: {medium_cardinality_cols[col]['variance']:.1f})")
    
    # =============================================================================
    # 4. GEOGRAPHIC FEATURE ENGINEERING
    # =============================================================================
    print("🌍 Processing Geographic Features:")
    
    # Keep original coordinates
    geographic_features = ['latitude', 'longitude']
    
    # Add derived geographic features
    if 'latitude' in df_encoded.columns and 'longitude' in df_encoded.columns:
        # Distance from conflict center (approximate center of your data)
        center_lat = df_encoded['latitude'].median()
        center_lon = df_encoded['longitude'].median()
        
        df_encoded['distance_from_center'] = np.sqrt(
            (df_encoded['latitude'] - center_lat)**2 + 
            (df_encoded['longitude'] - center_lon)**2
        )
        
        # Regional density (events per geographic area)
        df_encoded['lat_rounded'] = df_encoded['latitude'].round(1)
        df_encoded['lon_rounded'] = df_encoded['longitude'].round(1)
        location_counts = df_encoded.groupby(['lat_rounded', 'lon_rounded']).size()
        df_encoded['location_density'] = df_encoded.set_index(['lat_rounded', 'lon_rounded']).index.map(location_counts)
        
        # Clean up temporary columns
        df_encoded.drop(['lat_rounded', 'lon_rounded'], axis=1, inplace=True)
        
        geographic_features.extend(['distance_from_center', 'location_density'])
        print(f"   ✓ Added: {['distance_from_center', 'location_density']}")
    
    # =============================================================================
    # 5. PREPARE FINAL FEATURE SET
    # =============================================================================
    
    # Keep numeric target and features
    numeric_cols = [target_col] + [col for col in df_encoded.columns if df_encoded[col].dtype in ['int64', 'float64']]
    
    # Remove original categorical columns that were encoded
    original_categorical = [
        'disorder_type', 'event_type', 'sub_event_type', 'inter1', 'inter2',
        'admin2', 'admin3', 'actor1', 'actor2', 'interaction', 'admin1'
    ]
    
    final_features = [col for col in df_encoded.columns if col not in original_categorical]
    df_final = df_encoded[final_features].copy()
    
    # =============================================================================
    # 6. FINAL SUMMARY
    # =============================================================================
    print(f"\n✅ PREPROCESSING COMPLETE!")
    print(f"📈 Final dataset shape: {df_final.shape}")
    print(f"🎯 Target column: {target_col}")
    print(f"🔢 Feature columns: {df_final.shape[1] - 1}")
    
    print(f"\nFeature types created:")
    onehot_features = len([col for col in df_final.columns if any(prefix in col for prefix in onehot_cols)])
    target_features = len([col for col in df_final.columns if 'target_encoded' in col])
    label_features = len([col for col in df_final.columns if 'label_encoded' in col])
    
    print(f"   One-hot encoded: {onehot_features}")
    print(f"   Target encoded: {target_features}")
    print(f"   Label encoded: {label_features}")
    print(f"   Geographic: {len(geographic_features)}")
    
    # Check for any remaining issues
    print(f"\n🔍 Data quality check:")
    print(f"   Missing values: {df_final.isnull().sum().sum()}")
    print(f"   Infinite values: {np.isinf(df_final.select_dtypes(include=[np.number])).sum().sum()}")
    
    return df_final, label_encoders

def prepare_train_test_split(df_final, target_col='fatalities', test_size=0.2, random_state=42):
    """
    Prepare features and target for XGBoost training
    """
    from sklearn.model_selection import train_test_split
    
    # Separate features and target
    X = df_final.drop(columns=[target_col])
    y = df_final[target_col]
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=None
    )
    
    print(f"🔄 Train-Test Split:")
    print(f"   Training set: {X_train.shape[0]:,} samples")
    print(f"   Test set: {X_test.shape[0]:,} samples")
    print(f"   Features: {X_train.shape[1]}")
    
    # Check target distribution
    print(f"\n📊 Target distribution (fatalities):")
    print(f"   Train mean: {y_train.mean():.2f} ± {y_train.std():.2f}")
    print(f"   Test mean: {y_test.mean():.2f} ± {y_test.std():.2f}")
    print(f"   Min: {y.min()}, Max: {y.max()}")
    
    return X_train, X_test, y_train, y_test

# =============================================================================
# USAGE EXAMPLE
# =============================================================================

# Run the complete preprocessing pipeline:
df_ready, encoders = prepare_xgboost_features(df)

# Prepare for model training:
X_train, X_test, y_train, y_test = prepare_train_test_split(df_ready)

# Now ready for XGBoost!
import xgboost as xgb

# Create XGBoost dataset
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train model
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

model = xgb.train(params, dtrain, num_boost_round=100, 
                  evals=[(dtrain, 'train'), (dtest, 'test')],
                  early_stopping_rounds=10, verbose_eval=True)

In [32]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import warnings
warnings.filterwarnings('ignore')

def evaluate_model_performance(model, X_test, y_test, model_name="Model"):
    """Comprehensive model evaluation"""
    
    # Make predictions
    if hasattr(model, 'predict'):
        y_pred = model.predict(X_test)
    else:  # XGBoost DMatrix case
        dtest = xgb.DMatrix(X_test)
        y_pred = model.predict(dtest)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Additional metrics
    mape = np.mean(np.abs((y_test - y_pred) / np.maximum(y_test, 1))) * 100  # Avoid division by zero
    
    print(f"\n📊 {model_name} Performance:")
    print(f"   RMSE: {rmse:.3f}")
    print(f"   MAE:  {mae:.3f}")
    print(f"   R²:   {r2:.3f}")
    print(f"   MAPE: {mape:.2f}%")
    
    # Residual analysis
    residuals = y_test - y_pred
    print(f"   Mean residual: {residuals.mean():.3f}")
    print(f"   Std residuals: {residuals.std():.3f}")
    
    return {'rmse': rmse, 'mae': mae, 'r2': r2, 'mape': mape}

def quick_hyperparameter_tuning(X_train, y_train, X_test, y_test, n_trials=50):
    """
    Fast randomized search for hyperparameter tuning
    """
    print("🔍 Starting Quick Hyperparameter Tuning (Randomized Search)")
    print("-" * 60)
    
    # Define parameter distribution for randomized search
    param_dist = {
        'n_estimators': [100, 200, 300, 500, 800, 1000],
        'max_depth': [3, 4, 5, 6, 7, 8, 10],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15, 0.2, 0.3],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [1, 3, 5, 7, 10],
        'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
        'reg_alpha': [0, 0.001, 0.01, 0.1, 1, 10],
        'reg_lambda': [0, 0.001, 0.01, 0.1, 1, 10]
    }
    
    # XGBoost regressor
    xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1
    )
    
    # Randomized search
    start_time = time.time()
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_dist,
        n_iter=n_trials,
        scoring='neg_root_mean_squared_error',
        cv=5,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit
    random_search.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    print(f"⏱️ Training completed in {training_time:.1f} seconds")
    print(f"🎯 Best CV Score (RMSE): {-random_search.best_score_:.3f}")
    
    # Test set evaluation
    best_model = random_search.best_estimator_
    test_metrics = evaluate_model_performance(best_model, X_test, y_test, "Quick Tuned XGBoost")
    
    print(f"\n🏆 Best Parameters:")
    for param, value in random_search.best_params_.items():
        print(f"   {param}: {value}")
    
    return best_model, random_search.best_params_, test_metrics

def advanced_hyperparameter_tuning(X_train, y_train, X_test, y_test, base_params=None):
    """
    Advanced sequential hyperparameter tuning with GridSearchCV
    """
    print("🚀 Starting Advanced Hyperparameter Tuning (Sequential Grid Search)")
    print("-" * 70)
    
    if base_params is None:
        # Start with reasonable defaults
        current_params = {
            'objective': 'reg:squarederror',
            'random_state': 42,
            'n_jobs': -1
        }
    else:
        current_params = base_params.copy()
    
    best_score = float('inf')
    best_model = None
    tuning_history = []
    
    # Step 1: Tune n_estimators and learning_rate
    print("\n1️⃣ Tuning n_estimators and learning_rate...")
    param_grid_1 = {
        'n_estimators': [200, 500, 800, 1000, 1500],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15]
    }
    
    grid_search_1 = GridSearchCV(
        xgb.XGBRegressor(**current_params),
        param_grid_1,
        scoring='neg_root_mean_squared_error',
        cv=5,
        n_jobs=-1
    )
    grid_search_1.fit(X_train, y_train)
    
    current_params.update(grid_search_1.best_params_)
    current_score = -grid_search_1.best_score_
    print(f"   Best RMSE: {current_score:.3f}")
    print(f"   Best params: {grid_search_1.best_params_}")
    
    if current_score < best_score:
        best_score = current_score
        best_model = grid_search_1.best_estimator_
    
    tuning_history.append(('n_estimators + learning_rate', current_score, grid_search_1.best_params_))
    
    # Step 2: Tune tree structure
    print("\n2️⃣ Tuning tree structure (max_depth, min_child_weight)...")
    param_grid_2 = {
        'max_depth': [3, 4, 5, 6, 7, 8],
        'min_child_weight': [1, 3, 5, 7, 10]
    }
    
    grid_search_2 = GridSearchCV(
        xgb.XGBRegressor(**current_params),
        param_grid_2,
        scoring='neg_root_mean_squared_error',
        cv=5,
        n_jobs=-1
    )
    grid_search_2.fit(X_train, y_train)
    
    current_params.update(grid_search_2.best_params_)
    current_score = -grid_search_2.best_score_
    print(f"   Best RMSE: {current_score:.3f}")
    print(f"   Best params: {grid_search_2.best_params_}")
    
    if current_score < best_score:
        best_score = current_score
        best_model = grid_search_2.best_estimator_
    
    tuning_history.append(('Tree structure', current_score, grid_search_2.best_params_))
    
    # Step 3: Tune sampling parameters
    print("\n3️⃣ Tuning sampling (subsample, colsample_bytree)...")
    param_grid_3 = {
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.6, 0.8, 1.0]
    }
    
    grid_search_3 = GridSearchCV(
        xgb.XGBRegressor(**current_params),
        param_grid_3,
        scoring='neg_root_mean_squared_error',
        cv=5,
        n_jobs=-1
    )
    grid_search_3.fit(X_train, y_train)
    
    current_params.update(grid_search_3.best_params_)
    current_score = -grid_search_3.best_score_
    print(f"   Best RMSE: {current_score:.3f}")
    print(f"   Best params: {grid_search_3.best_params_}")
    
    if current_score < best_score:
        best_score = current_score
        best_model = grid_search_3.best_estimator_
    
    tuning_history.append(('Sampling', current_score, grid_search_3.best_params_))
    
    # Step 4: Tune regularization
    print("\n4️⃣ Tuning regularization (gamma, reg_alpha, reg_lambda)...")
    param_grid_4 = {
        'gamma': [0, 0.1, 0.2, 0.3, 0.5],
        'reg_alpha': [0, 0.001, 0.01, 0.1, 1],
        'reg_lambda': [0, 0.001, 0.01, 0.1, 1]
    }
    
    grid_search_4 = GridSearchCV(
        xgb.XGBRegressor(**current_params),
        param_grid_4,
        scoring='neg_root_mean_squared_error',
        cv=5,
        n_jobs=-1
    )
    grid_search_4.fit(X_train, y_train)
    
    current_params.update(grid_search_4.best_params_)
    current_score = -grid_search_4.best_score_
    print(f"   Best RMSE: {current_score:.3f}")
    print(f"   Best params: {grid_search_4.best_params_}")
    
    if current_score < best_score:
        best_score = current_score
        best_model = grid_search_4.best_estimator_
    
    tuning_history.append(('Regularization', current_score, grid_search_4.best_params_))
    
    # Final evaluation
    print(f"\n🎯 ADVANCED TUNING COMPLETE!")
    print(f"   Best CV RMSE: {best_score:.3f}")
    
    final_metrics = evaluate_model_performance(best_model, X_test, y_test, "Advanced Tuned XGBoost")
    
    # Show tuning progression
    print(f"\n📈 Tuning Progression:")
    for step, score, params in tuning_history:
        print(f"   {step}: {score:.3f}")
    
    # Final parameters
    print(f"\n🏆 Final Optimized Parameters:")
    final_params = best_model.get_params()
    for param, value in sorted(final_params.items()):
        if param not in ['n_jobs', 'random_state', 'objective']:
            print(f"   {param}: {value}")
    
    return best_model, current_params, final_metrics, tuning_history

def bayesian_optimization_tuning(X_train, y_train, X_test, y_test, n_trials=100):
    """
    Bayesian optimization using Optuna (if available)
    """
    try:
        import optuna
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        print("🧠 Starting Bayesian Optimization with Optuna")
        print("-" * 50)
        
        def objective(trial):
            # Define hyperparameter search space
            params = {
                'objective': 'reg:squarederror',
                'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                'gamma': trial.suggest_float('gamma', 0, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0001, 100, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0001, 100, log=True),
                'random_state': 42,
                'n_jobs': -1
            }
            
            # Cross-validation
            model = xgb.XGBRegressor(**params)
            scores = cross_val_score(model, X_train, y_train, 
                                   scoring='neg_root_mean_squared_error', cv=5)
            return -scores.mean()  # Optuna minimizes
        
        # Create study and optimize
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=n_trials)
        
        # Train best model
        best_params = study.best_params.copy()
        best_params.update({'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': -1})
        
        best_model = xgb.XGBRegressor(**best_params)
        best_model.fit(X_train, y_train)
        
        print(f"🎯 Best CV Score (RMSE): {study.best_value:.3f}")
        
        # Evaluate on test set
        final_metrics = evaluate_model_performance(best_model, X_test, y_test, "Bayesian Optimized XGBoost")
        
        print(f"\n🏆 Best Parameters (Bayesian Optimization):")
        for param, value in study.best_params.items():
            print(f"   {param}: {value}")
        
        return best_model, best_params, final_metrics
        
    except ImportError:
        print("❌ Optuna not installed. Install with: pip install optuna")
        print("   Falling back to advanced grid search...")
        return advanced_hyperparameter_tuning(X_train, y_train, X_test, y_test)

def compare_tuning_methods(X_train, y_train, X_test, y_test):
    """
    Compare all tuning methods and select the best one
    """
    print("🏁 COMPREHENSIVE HYPERPARAMETER TUNING COMPARISON")
    print("=" * 80)
    
    results = {}
    
    # Method 1: Quick Random Search
    print("\n" + "="*50)
    print("METHOD 1: QUICK RANDOM SEARCH")
    print("="*50)
    start_time = time.time()
    model_1, params_1, metrics_1 = quick_hyperparameter_tuning(X_train, y_train, X_test, y_test)
    time_1 = time.time() - start_time
    results['Quick Random Search'] = {
        'model': model_1, 'params': params_1, 'metrics': metrics_1, 'time': time_1
    }
    
    # Method 2: Advanced Sequential Grid Search
    print("\n" + "="*50)
    print("METHOD 2: ADVANCED SEQUENTIAL GRID SEARCH")
    print("="*50)
    start_time = time.time()
    model_2, params_2, metrics_2, history_2 = advanced_hyperparameter_tuning(
        X_train, y_train, X_test, y_test, base_params=params_1
    )
    time_2 = time.time() - start_time
    results['Advanced Grid Search'] = {
        'model': model_2, 'params': params_2, 'metrics': metrics_2, 'time': time_2
    }
    
    # Method 3: Bayesian Optimization (if available)
    print("\n" + "="*50)
    print("METHOD 3: BAYESIAN OPTIMIZATION")
    print("="*50)
    start_time = time.time()
    try:
        model_3, params_3, metrics_3 = bayesian_optimization_tuning(X_train, y_train, X_test, y_test)
        time_3 = time.time() - start_time
        results['Bayesian Optimization'] = {
            'model': model_3, 'params': params_3, 'metrics': metrics_3, 'time': time_3
        }
    except Exception as e:
        print(f"Bayesian optimization failed: {e}")
        time_3 = 0
    
    # Final comparison
    print("\n" + "="*80)
    print("🏆 FINAL COMPARISON")
    print("="*80)
    
    comparison_df = pd.DataFrame({
        method: {
            'Test RMSE': data['metrics']['rmse'],
            'Test MAE': data['metrics']['mae'],
            'Test R²': data['metrics']['r2'],
            'Time (min)': data['time'] / 60
        }
        for method, data in results.items()
    }).T
    
    print(comparison_df.round(3))
    
    # Select best model
    best_method = comparison_df['Test RMSE'].idxmin()
    best_model = results[best_method]['model']
    
    print(f"\n🥇 WINNER: {best_method}")
    print(f"   Best Test RMSE: {results[best_method]['metrics']['rmse']:.3f}")
    print(f"   Training time: {results[best_method]['time']/60:.1f} minutes")
    
    return best_model, results[best_method]['params'], results

# =============================================================================
# USAGE EXAMPLE
# =============================================================================

# After running your preprocessing pipeline:
X_train, X_test, y_train, y_test = prepare_train_test_split(df_ready)

# Option 1: Quick tuning (5-10 minutes)
best_model_quick, best_params_quick, metrics_quick = quick_hyperparameter_tuning(
    X_train, y_train, X_test, y_test, n_trials=50
)

# Option 2: Comprehensive comparison (30-60 minutes)
best_model, best_params, all_results = compare_tuning_methods(
    X_train, y_train, X_test, y_test
)

# Use the best model for predictions
final_predictions = best_model.predict(X_test)

🔄 Train-Test Split:
   Training set: 7,686 samples
   Test set: 1,922 samples
   Features: 33

📊 Target distribution (fatalities):
   Train mean: 5.10 ± 10.99
   Test mean: 5.32 ± 11.59
   Min: 1, Max: 213
🔍 Starting Quick Hyperparameter Tuning (Randomized Search)
------------------------------------------------------------
Fitting 5 folds for each of 50 candidates, totalling 250 fits
⏱️ Training completed in 31.0 seconds
🎯 Best CV Score (RMSE): 6.205

📊 Quick Tuned XGBoost Performance:
   RMSE: 6.370
   MAE:  2.464
   R²:   0.698
   MAPE: 80.95%
   Mean residual: 0.294
   Std residuals: 6.365

🏆 Best Parameters:
   subsample: 1.0
   reg_lambda: 1
   reg_alpha: 1
   n_estimators: 100
   min_child_weight: 7
   max_depth: 3
   learning_rate: 0.15
   gamma: 0.5
   colsample_bytree: 0.7
   colsample_bylevel: 1.0
🏁 COMPREHENSIVE HYPERPARAMETER TUNING COMPARISON

METHOD 1: QUICK RANDOM SEARCH
🔍 Starting Quick Hyperparameter Tuning (Randomized Search)
-----------------------------------------